In [2]:
from network_evaluation_tools import gene_conversion_tools as gct
from network_evaluation_tools import data_import_tools as dit
import pandas as pd

In [1]:
wd = '/cellar/users/snwright/Data/Network_Analysis/'

## Load HPRD Raw Data
#### Source: http://www.hprd.org/download
#### The file requires registration with the database. Download the file: HPRD_Release9_041310.tar.gz
Downloaded: August 12, 2016  
Last Updated: June 29, 2010  
The following files are manipulated after unzipping the .tar.gz file

In [13]:
HPRD_Raw = pd.read_csv(wd+'Network_Data_Raw/HPRD_cleaned.txt',sep='\t',header=None)

,0,1,2,3,4,5,6,7
0,ALDH1A1,1,NP_000680.2,ALDH1A1,1,NP_000680.2,in vivo;yeast 2-hybrid,"12081471,16189514"
1,ITGA7,2761,NP_001138468.1,CHRNA1,7,NP_001034612.1,in vivo,10910772
2,PPP1R9A,16000,NP_060120.2,ACTG1,17,NP_001605.1,in vitro;in vivo,"9362513,12052877"
3,SRGN,1513,NP_002718.2,CD44,115,NP_000601.3,in vivo,9334256
4,GRB7,3311,NP_005301.2,ERBB2,1281,NP_004439.2,in vitro;in vivo,9079677


In [9]:
# Assign column names from README file from archive
HPRD_Raw.columns = ['Interactor 1 Gene Symbol', 'Interactor 1 HPRD ID', 'Interactor 1 RefSeq ID',
                    'Interactor 2 Gene Symbol', 'Interactor 2 HPRD ID', 'Interactor 2 RefSeq ID',
                    'Experiment Type', 'PubMed ID']

In [10]:
# Convert table of interactions to edgelist (no scores given)
# Also no gene symbol conversion necessary because network is given in symbol format already
HPRD_edgelist = HPRD_Raw[['Interactor 1 Gene Symbol', 'Interactor 2 Gene Symbol']].values.tolist()
print('Edges in HPRD:', len(HPRD_edgelist))

Edges in HPRD: 39240


In [11]:
# Sort each edge representation for filtering
HPRD_edgelist_sorted = [sorted(edge) for edge in HPRD_edgelist]

In [14]:
# Original
# 39240 input edges
# 2160 self-edges removed
# 0 edges with un-mapped genes removed
# 41 duplicate edges removed
# Edge list filtered: 0.05 seconds
# 37039 Edges remaining
# Filter edgelist for duplicate nodes and for self-edges
HPRD_edgelist_filt = gct.filter_converted_edgelist(pd.DataFrame(HPRD_edgelist_sorted, columns=["symbol_n1", "symbol_n2"]))

39240 input edges
2160 self-edges removed
0 edges with un-mapped genes removed
41 duplicate edges removed
Edge list filtered: 0.04 seconds
37039 Edges remaining


In [15]:
# Save genelist to file
gct.write_edgelist(HPRD_edgelist_filt, wd+'Processed_Data/Network_SIFs_Symbol/HPRD_Symbol_2010.sif')

Edge list saved: 0.09 seconds
